In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt


def gen_uniform_amp(amp=1.0, xn=10000):
    """Generates uniform random data between
    -amp and +amp
    and of length xn
    Arguments:
        amp: maximum/minimum range of uniform data
        xn: length of series
    """
    data_input = np.random.uniform(-1 * amp, +1 * amp, xn)
    data_input = pd.DataFrame(data_input)
    return data_input


def create_model(stateful, lahead, batch_size):
    model = Sequential()
    model.add(LSTM(20,
              input_shape=(lahead, 1),
              batch_size=batch_size,
              stateful=stateful))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model


def split_data(x, y, input_len, batch_size, ratio=0.8):
    to_train = int(input_len * ratio)
    # tweak to match with batch_size
    to_train -= to_train % batch_size

    x_train = x[:to_train]
    y_train = y[:to_train]
    x_test = x[to_train:]
    y_test = y[to_train:]

    # tweak to match with batch_size
    to_drop = x.shape[0] % batch_size
    if to_drop > 0:
        x_test = x_test[:-1 * to_drop]
        y_test = y_test[:-1 * to_drop]

    # some reshaping
    reshape_3 = lambda x: x.values.reshape((x.shape[0], x.shape[1], 1))
    x_train = reshape_3(x_train)
    x_test = reshape_3(x_test)

    reshape_2 = lambda x: x.values.reshape((x.shape[0], 1))
    y_train = reshape_2(y_train)
    y_test = reshape_2(y_test)

    return (x_train, y_train), (x_test, y_test)


def main(input_len=1000, tsteps=2, lahead=1, batch_size=1, epochs=10):
    np.random.seed(1986)
    print('Generating Data...')
    to_drop = max(tsteps - 1, lahead - 1)
    data_input = gen_uniform_amp(amp=0.1, xn=input_len + to_drop)
    # set the target to be a N-point average of the input
    expected_output = data_input.rolling(window=tsteps, center=False).mean()
    if lahead > 1:
        data_input = np.repeat(data_input.values, repeats=lahead, axis=1)
        data_input = pd.DataFrame(data_input)
        for i, c in enumerate(data_input.columns):
            data_input[c] = data_input[c].shift(i)
    # drop the nan
    expected_output = expected_output[to_drop:]
    data_input = data_input[to_drop:]
    print('Creating Stateful Model...')
    model_stateful = create_model(stateful=True, lahead=lahead, batch_size=batch_size)
    (x_train, y_train), (x_test, y_test) = split_data(data_input, expected_output, input_len, batch_size)
    print('Training Stateful Model...')
    for i in range(epochs):
        model_stateful.fit(x_train,
                           y_train,
                           batch_size=batch_size,
                           epochs=1,
                           verbose=1,
                           validation_data=(x_test, y_test),
                           shuffle=False)
        model_stateful.reset_states()
    predicted_stateful = model_stateful.predict(x_test, batch_size=batch_size)

    print('Creating Stateless Model...')
    model_stateless = create_model(stateful=False, lahead=lahead, batch_size=batch_size)

    print('Training Stateless Model...')
    model_stateless.fit(x_train,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test),
                        shuffle=False)
    predicted_stateless = model_stateless.predict(x_test, batch_size=batch_size)
    # RMSE
    stateless_rmse = sqrt(mean_squared_error(y_test, predicted_stateless))
    stateful_rmse = sqrt(mean_squared_error(y_test, predicted_stateful))
    print('>>> Stateful RMSE: %f' % stateful_rmse)
    print('>>> Stateless RMSE: %f' % stateless_rmse)

Using TensorFlow backend.


In [2]:
main(input_len=1000, tsteps=2, lahead=1, batch_size=1, epochs=10)

Generating Data...
Creating Stateful Model...
Training Stateful Model...
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 2ms/step - loss: 6.0338e-04 - val_loss: 5.2130e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 2ms/step - loss: 3.0422e-04 - val_loss: 2.6662e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 2.4787e-04 - val_loss: 2.2040e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 2.4690e-04 - val_loss: 2.1002e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 2.5166e-04 - val_loss: 2.0341e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 2.4603e-04 - val_loss: 2.0273e-04
Train on 

In [3]:
main(input_len=1000, tsteps=2, lahead=2, batch_size=1, epochs=10)

Generating Data...
Creating Stateful Model...
Training Stateful Model...
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s 4ms/step - loss: 4.2033e-04 - val_loss: 2.6419e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s 3ms/step - loss: 1.7813e-04 - val_loss: 1.4733e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s 3ms/step - loss: 1.0431e-04 - val_loss: 1.1480e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 1.0678e-04 - val_loss: 1.3304e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 3s 3ms/step - loss: 8.9428e-05 - val_loss: 6.7338e-05
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 8.3703e-05 - val_loss: 0.0021
Train on 800 

In [4]:
main(input_len=1000, tsteps=2, lahead=2, batch_size=2, epochs=10)

Generating Data...
Creating Stateful Model...
Training Stateful Model...
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 2ms/step - loss: 5.7222e-04 - val_loss: 3.3209e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 2.7512e-04 - val_loss: 2.5846e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 2.1867e-04 - val_loss: 2.0454e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 1.6616e-04 - val_loss: 1.4092e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 1.2200e-04 - val_loss: 1.4021e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 1.3346e-04 - val_loss: 1.2911e-04
Train on 

In [5]:
main(input_len=1000, tsteps=2, lahead=1, batch_size=2, epochs=10)

Generating Data...
Creating Stateful Model...
Training Stateful Model...
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 2s 3ms/step - loss: 0.0012 - val_loss: 0.0012
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 0.0011 - val_loss: 0.0010
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 0.0010 - val_loss: 9.6194e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 9.5830e-04 - val_loss: 9.1806e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 9.2466e-04 - val_loss: 8.9051e-04
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 1s 1ms/step - loss: 9.0136e-04 - val_loss: 8.7217e-04
Train on 800 samples, validat